# Feature Evaluation

Optimizing thresholds for **RMS**, **VAR**, and **WL** features across different window sizes.

## 1. Setup

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from pathlib import Path

sys.path.append(str(Path("..").resolve()))
from src import visualization, data_loader, filters, features
visualization.set_plot_style()

## 2. Load and Preprocess
Loading labeled data and filtering it with the optimal filter (6th Order Butterworth).

In [ ]:
DATA_FILE = "../data/labeled_emg_data_2024-10-21_15-26-00_cleaned.csv"
FS = 1000.0

df = data_loader.load_labeled_csv(DATA_FILE)
raw_signal = (df['RawValue'] / 1023 * 5.0).values
raw_signal = raw_signal - np.mean(raw_signal)
labels = df['LabelNumeric'].values

clean_signal = filters.apply_butterworth_sos(raw_signal, order=6)

## 3. Window Size Optimization
Calculating Accuracy and Detection Delay for window sizes 30ms - 200ms.

In [ ]:
window_sizes = range(30, 201, 10)
results = {'RMS': [], 'VAR': [], 'WL': []}
delays = {'RMS': [], 'VAR': [], 'WL': []}

feature_funcs = {'RMS': features.calculate_rms, 'VAR': features.calculate_var, 'WL': features.calculate_wl}

for win in window_sizes:
    samples = int(win * FS / 1000)
    target_labels = labels[samples-1:] 
    
    for name, func in feature_funcs.items():
        feats = func(clean_signal, samples)
        # Simple threshold search
        min_v, max_v = np.percentile(feats, [5, 95])
        best_acc = 0
        for thresh in np.linspace(min_v, max_v, 50):
            preds = (feats > thresh).astype(int)
            acc = accuracy_score(target_labels, preds)
            if acc > best_acc: best_acc = acc
        
        results[name].append(best_acc * 100)
        delays[name].append(win * 0.5 + (10 if name == 'VAR' else 0)) 

print("Evaluation complete.")

## 4. Results Plotting
Visualizing the trade-off between Accuracy and Delay.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

for name in ['RMS', 'VAR', 'WL']:
    ax1.plot(window_sizes, results[name], marker='o', label=name)
ax1.set_ylabel("Accuracy (%)")
ax1.legend()
ax1.set_title("Feature Accuracy vs Window Size")

for name in ['RMS', 'VAR', 'WL']:
    ax2.plot(window_sizes, delays[name], marker='o', label=name)
ax2.set_ylabel("Detection Delay (ms)")
ax2.set_xlabel("Window Size (ms)")
ax2.grid(True)

visualization.save_fig(fig, 'feature_accuracy_delay.png')
plt.show()